In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pydrake.all import (
    ModelVisualizer,
    StartMeshcat,
    Simulator,
)
import pydot

from IPython.display import SVG, display
from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario

import sponana.utils

In [3]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [5]:
inspect_spot_model = False

if inspect_spot_model:
    # Inspecting the Spot Model
    visualizer = ModelVisualizer(meshcat=meshcat)
    sponana.utils.configure_parser(visualizer.parser())
    visualizer.AddModels(url="package://manipulation/spot/spot_with_arm.urdf")
    visualizer.Run(loop_once=not running_as_notebook)
    meshcat.DeleteAddedControls()

In [19]:
def create_scene(scenario_data: str):
    # Clean up the Meshcat instance.
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    scenario = load_scenario(data=scenario_data)
    station = MakeHardwareStation(
        scenario, meshcat, package_xmls=sponana.utils.PACKAGE_XMLS
    )

    return station

In [20]:
def initialize_simulation(diagram):
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.0)
    return simulator


def run_simulation(scenario_data):
    station = create_scene(scenario_data)
    simulator = initialize_simulation(station)
    meshcat.StartRecording()
    finish_time = 2.0
    simulator.AdvanceTo(finish_time)
    meshcat.PublishRecording()

In [9]:
scenario_data = """
cameras:
    main_camera:
        name: camera0
        depth: True
        X_PB:
            base_frame: world
            translation: [0, 0, 0.8]
            rotation: !Rpy { deg: [180, 0, 90]}
directives:
- add_model:
    name: spot
    file: package://manipulation/spot/spot_with_arm.urdf
    
- add_model:
    name: banana
    file: package://sponana/banana/banana.sdf

- add_model:
    name: floor
    file: package://sponana/platform.sdf

- add_weld:
    parent: world
    child: floor::platform_center
    X_PC:
        translation: [0, 0, -.5]
"""

In [45]:
run_simulation(scenario_data)


ERROR: In vtkPNGReader.cxx, line 132
(nullptr): Unknown file type! Not a PNG file!

ERROR: In vtkPNGReader.cxx, line 270
vtkPNGReader (0x2bdad8a60): Invalid file header: not a PNG file

ERROR: In vtkPNGReader.cxx, line 132
(nullptr): Unknown file type! Not a PNG file!

ERROR: In vtkPNGReader.cxx, line 394
vtkPNGReader (0x2bdad8a60): Invalid file header: not a PNG file



In [ ]:
diagram = create_scene(scenario_data)
display(
    SVG(
        pydot.graph_from_dot_data(diagram.GetGraphvizString(max_depth=1))[
            0
        ].create_svg()
    )
)